<a href="https://colab.research.google.com/github/Jyothika083/Resume_Parser/blob/main/Mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 21.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!nvidia-smi

Wed May  8 11:24:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 22.29 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [5]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', 'r'))

In [6]:
len(cv_data)

200

In [7]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [9]:
def get_spacy_doc(file,data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entitiy_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entitiy_indices:
          skip_entity = True
          break
      if skip_entity ==True:
        continue

      entitiy_indices = entitiy_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) +"  "+ str(text)+ "\n"
        file.write(err_data)
      else:
        ents.append(span)
    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
  return db





In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size = 0.3)

In [11]:
file = open('error.txt','w', encoding='utf-8')
db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')
db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 60/60 [00:01<00:00, 59.26it/s]


In [12]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 127kB/s]
config.json: 100% 481/481 [00:00<00:00, 2.77MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 14.8MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 50.8MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 55.2MB/s]
model.safetensors: 100% 499M/499M [00:03<00:00, 154MB/s]
Some weights of R

In [13]:
nlp=spacy.load('/content/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [16]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 34.9 MB/s eta 0:00:00


In [17]:
import sys, fitz

In [18]:
fname = '/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)

In [19]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [20]:
text = text.strip()

In [21]:
text = ' '.join(text.split())

In [22]:
doc =nlp(text)
for ent in doc.ents:
  print(ent.text, "--------->" , ent.label_)

Alice Clark ---------> Name
AI / Machine Learning Delhi ---------> Designation
20+ years ---------> Years of Experience
Software Engineer ---------> Designation
Microsoft ---------> Companies worked at
Indian Institute of Technology ---------> College Name
2001 ---------> Graduation Year
Machine Learning, Natural Language Processing, and Big Data Handling ADDITIONAL INFORMATION Professional Skills • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal skills with ability to interact with individuals at all the levels • Quick learner and maintains cordial relationship with project manager and team members and good performer both in team and independent job environments • Positive attitude towards superiors &amp; peers • Supervised junior developers throughout project lifecycle and provided technical assistance ---------> Skills


# Domain prediction

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
df=pd.read_csv("/content/drive/MyDrive/UpdatedResumeDataSet.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [29]:
import re
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText

In [30]:
df['Resume']=df['Resume'].apply(lambda x:cleanResume(x))

In [31]:
#classification problem
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [32]:
le.fit(df['Category'])
df['Category']=le.transform(df['Category'])

In [33]:
df

,Category,Resume
0,6,Skills Programming Languages Python pandas num...
1,6,Education Details May 2013 to May 2017 B E UIT...
2,6,Areas of Interest Deep Learning Control System...
3,6,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,6,Education Details MCA YMCAUST Faridabad Haryan...
...,...,...
957,23,Computer Skills Proficient in MS office Word B...
958,23,Willingness to a ept the challenges Positive ...
959,23,PERSONAL SKILLS Quick learner Eagerness to lea...
960,23,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')
tfidf.fit(df['Resume'])
requiredText=tfidf.transform(df['Resume'])

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train,X_test,y_train,y_test=train_test_split(requiredText,df['Category'],test_size=0.2,random_state=42)

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
clf=OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train,y_train)
ypred=clf.predict(X_test)
print(accuracy_score(y_test,ypred))

0.9844559585492227


In [39]:
import pickle
pickle.dump(tfidf,open('tfidf.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))

In [40]:
myresume="""Vrishank J Vasist
E-mail:
vrishank.cs21@bmsce.ac.in
 Phone: 8870066818
Objectives:
➢ To strive for excellence, to work in an environment that will enhance my knowledge and career, work hard with full
dedication and to achieve personal and organizational goals.
Education
➢ 2021 – 2025:  Computer Science at BMS College of Engineering, Bull Temple Road, Bangalore, Karnataka.
➢ 2019 - 2021: 91%. Pre-University College - Sri Chaitanya PU College, Koramangala, Bangalore, Karnataka.
➢ 2018 - 2019: S.S.L.C at Sri Chaitanya Techno School, Electronic City, Bangalore, Karnataka. Passed out with 92.8%.
Skills and Certifications
➢ Technical Skills: Python, Django, SQL, JavaScript, Java, C
➢ Tools and Technologies: Git, VS Code, Jupyter Notebook, Google Colab, Bootstrap.
➢ Certifications: Azure AI Fundamentals, Azure Fundamentals, Udemy Python Course, Udemy
JavaScript Course.
Work Experience:
➢ Technical Internship at BMS College of Engineering. Researched Image Classification and Recognition
using Neural Networks and ML under Assistant Professor Sneha S Bagalkot.
➢ 6 Month Virtual Internship from Samsung Prism working on Image Enhancement using Neural
Networks and ML.
Projects:
➢ Discussion Forum: Fully Functional Discussion Website made using Django and stylized using
Bootstrap.
➢ Image Classification Application: Machine Learning CNN Model created using TensorFlow.
Classifies objects into three categories.
➢ Fitness Application: Application made completely in Java consisting of various features such as
Water Tracking, Calorie Counting and many calculators.
Personal Details
➢ Name: Vrishank J Vasist
➢ Date of Birth: 8th April 2003
➢ Nationality: Indian
➢ Languages: English, Kannada, Hindi, Tamil
Declaration
I hereby declare that the details furnished above are true and correct to the best of my knowledge.
Date: 6th November 2023
Place: Bangalore, Karnataka """

In [41]:
import pickle

#load the trained classifier
clf=pickle.load(open('clf.pkl','rb'))

#clean the input resume
cleanedResume=cleanResume(myresume)

#transform the cleaned resume using the trained tfidfvectorization
inputFeatures=tfidf.transform([cleanedResume])

#make the prediction using the loaded classifier
predictionId=clf.predict(inputFeatures)[0]

#map category ID to category name
categoryMapping = {
    15: "Java Developer",
    23: "Testing",
    8: "DevOps Engineer",
    20: "Python Developer",
    24: "Web Designing",
    12: "HR",
    13: "Hadoop",
    3: "Blockchain",
    10: "ETL Developer",
    18: "Operations Manager",
    6: "Data Science",
    22: "Sales",
    16: "Mechanical Engineer",
    1: "Arts",
    7: "Database",
    11: "Electrical Engineering",
    14: "Health and fitness",
    19: "PMO",
    4: "Business Analyst",
    9: "DotNet Developer",
    2: "Automation Testing",
    17: "Network Security Engineer",
    21: "SAP Developer",
    5: "Civil Engineer",
    0: "Advocate",
}
categoryName = categoryMapping.get(predictionId, "Unknown")

print("Predicted Category:", categoryName)
print(predictionId)


Predicted Category: Data Science
6


In [23]:
doc =nlp('''Harika N
B.E. in Computer Science Engineering harikatanguturi13@gmail.com
B.M.S College of Engineering, Bangalore 9206547808
CAREER OBJECTIVE:
An energetic, ambitious, and highly motivated individual seeking a challenging profession in
computer science that will allow me to convey my theoretical and practical skills while also allowing
me to advance in my technological interests.
EDUCATIONAL QUALIFICATIONS:
B.M.S College of Engineering 2021-2025
B.E. in CSE, CGPA: 9.48 /10 Bangalore, India
MS Ramaiah Composite PU College 2019-2021
PUC (Class XII), Aggregate: 99.1 Bangalore, India
SJR Primary and High School 2018-2019
SSLC (Class X), Aggregate: 97.7 Bangalore, India
TECHNICAL SKILLS:
Languages: C, JavaScript, Java, HTML, CSS, SQL
Frameworks: Node.js, express.js
Database: MySQL, MongoDB, Firebase
Libraries: Numpy, Pandas, Matplotlib
Tools/Technologies: VS Code, Google Collab, GitHub, Docker Hub, Figma
PROJECTS:
Web Development Project
It is a Library Management System website constructed to handle the movement of books and
maintain records of library users using HTML, CSS, and JS in the frontend to connect MySQL and
hosting on GitHub, as well as Docker Hub and built CI/CD pipelining.
Mobile Application Project
It is Inventory Management App that enables to visually track items and any of their details including
quantity, price, condition, notes in a more intuitive way for a team to track the inventory across
multiple locations. It is built using figma , flutter and firebase.
R&D Intern
Studied papers on various CAPTCHA mechanisms and a survey research paper was written which
consisted of Skin detection sensor-based assistance for CAPTCHA which can overcome the
limitations of traditional CAPTCHA using Internet of Things.
DECLARATION:
I hereby declare that information given is true, complete and correct to the best of my knowledge and
belief.
Place: Bengaluru (Harika N)''')
for ent in doc.ents:
  print(ent.text, "--------->" , ent.label_)

Harika N ---------> Name
B.E. in Computer Science Engineering ---------> Degree
B.M.S College of Engineering ---------> College Name
B.M.S College of Engineering ---------> College Name
2021 ---------> Graduation Year
2025 ---------> Graduation Year
B.E. in CSE ---------> Degree
MS Ramaiah Composite PU College ---------> College Name
2019 ---------> Graduation Year
2021 ---------> Graduation Year
SJR Primary and High School 2018-2019
SSLC (Class X), Aggregate: 97.7 Bangalore, India ---------> College Name
Languages: C, JavaScript, Java, HTML, CSS, SQL
Frameworks: Node.js, express.js
Database: MySQL, MongoDB, Firebase
Libraries: Numpy, Pandas, Matplotlib
Tools/Technologies: VS Code, Google Collab, GitHub, Docker Hub, Figma
PROJECTS:
Web Development Project
It is a Library Management System website constructed to handle the movement of books and
maintain records of library users using HTML, CSS, and JS in the frontend to connect MySQL and
hosting on GitHub, as well as Docker Hub and buil